In [1]:
import semantic_kernel as sk

kernel = sk.Kernel()

from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

api_key, org_id = sk.openai_settings_from_dot_env()
service_id = "default"
kernel.add_service(
    OpenAIChatCompletion(service_id=service_id, ai_model_id="gpt-3.5-turbo-0125", api_key=api_key, org_id=org_id),
)

plugin = kernel.import_plugin_from_prompt_directory("./Plugins/", "ReportPlugin")
OutlineJSON_function = plugin["OutlineJSON"]

OutlineJSON = await kernel.invoke(OutlineJSON_function, sk.KernelArguments(input="Báo cáo thị trường Xăng dầu năm 2023"))
print(OutlineJSON)


{'report': {'title': 'Báo cáo thị trường Xăng dầu năm 2023','introduction': 'Báo cáo này sẽ đánh giá tình hình thị trường xăng dầu trong năm 2023, bao gồm các yếu tố ảnh hưởng và dự báo phát triển trong thời gian tới.','body': {'headings': ['1. Tình hình thị trường xăng dầu hiện tại','2. Các yếu tố ảnh hưởng đến giá xăng dầu','3. Dự báo phát triển thị trường trong năm 2023']}, 'conclusion': 'Tóm lại, thị trường xăng dầu trong năm 2023 đang đối diện với nhiều thách thức và cơ hội.','recommendations': 'Dựa trên những phân tích trong báo cáo, chúng tôi đưa ra các khuyến nghị cho các doanh nghiệp hoạt động trong lĩnh vực này.'}}


In [2]:
from Plugins.utils import json_split
outline_dict = json_split(str(OutlineJSON))
outline_dict

{'title': 'Báo cáo thị trường Xăng dầu năm 2023',
 'introduction': 'Báo cáo này sẽ đánh giá tình hình thị trường xăng dầu trong năm 2023, bao gồm các yếu tố ảnh hưởng và dự báo phát triển trong thời gian tới.',
 'body': {'headings': ['1. Tình hình thị trường xăng dầu hiện tại',
   '2. Các yếu tố ảnh hưởng đến giá xăng dầu',
   '3. Dự báo phát triển thị trường trong năm 2023']},
 'conclusion': 'Tóm lại, thị trường xăng dầu trong năm 2023 đang đối diện với nhiều thách thức và cơ hội.',
 'recommendations': 'Dựa trên những phân tích trong báo cáo, chúng tôi đưa ra các khuyến nghị cho các doanh nghiệp hoạt động trong lĩnh vực này.'}

In [3]:
Introduction_function = plugin["Introduction"]
Introduction = await kernel.invoke(Introduction_function, sk.KernelArguments(introduction=outline_dict["introduction"]
                                                                            , reportLayout = str(OutlineJSON)))
print(Introduction)

Báo cáo thị trường Xăng dầu năm 2023 sẽ cung cấp một cái nhìn tổng quan về tình hình thị trường xăng dầu trong năm 2023, bao gồm các yếu tố ảnh hưởng và dự báo phát triển trong thời gian tới. Báo cáo sẽ bao gồm ba phần chính: 1. Tình hình thị trường xăng dầu hiện tại, 2. Các yếu tố ảnh hưởng đến giá xăng dầu, và 3. Dự báo phát triển thị trường trong năm 2023. Cuối cùng, báo cáo sẽ kết luận với việc thị trường xăng dầu trong năm 2023 đang đối diện với nhiều thách thức và cơ hội, và đưa ra các khuyến nghị dựa trên những phân tích trong báo cáo cho các doanh nghiệp hoạt động trong lĩnh vực này.


In [15]:
from Plugins.google_search import GoogleSearchPlugin
from Plugins.utils import token_count
import asyncio
TOKEN_LIMIT = 10000

google_plugin = kernel.import_plugin_from_object(GoogleSearchPlugin(), plugin_name = "GoogleSearchPlugin")
google_search_function = google_plugin["GoogleSearch"]
get_content_function = google_plugin["GetContentFromURL"]

column_function = plugin["Column"]
column_summ_function = plugin["ColumnSumm"]
headings = outline_dict["body"]["headings"]

async def columnist(heading):
    search_result = []
    tokens = 0
    search_result_urls = await kernel.invoke(google_search_function, sk.KernelArguments(keyword=heading, num_results = 15))
    search_result_urls = search_result_urls.value
    while True:
        if not search_result_urls:
            break
        url = search_result_urls.pop(0)
        content = await kernel.invoke(get_content_function, sk.KernelArguments(url = url))
        content = content.value
        new_tokens = token_count(content)
        if tokens + new_tokens > TOKEN_LIMIT:
            print(tokens)
            if tokens >= TOKEN_LIMIT//2:
                break
            else:
                continue
        tokens += new_tokens
        search_result.append(content)
    search_result = " ".join(search_result)
    column = await kernel.invoke(column_function, sk.KernelArguments(input = search_result, body_content = heading))
    column_summ = await kernel.invoke(column_summ_function, sk.KernelArguments(input = search_result, body_content = heading))
    return str(column), str(column_summ)

coros = []
for heading in headings:
    coros.append(columnist(heading))

values = await asyncio.gather(*coros)
print(values)




Overwriting plugin "GoogleSearchPlugin" in collection


8662
8596
1826
8527
[('The current situation of the oil market is facing significant challenges due to the strong increase in global oil prices. The world oil market has been influenced by the rising demand as economies around the world are gradually recovering. Production and business activities are resuming as many countries are adopting living together, adapting safely to the Covid-19 pandemic. Energy demand, including oil, is also increasing to prepare for the winter season as gas prices surge. Additionally, OPEC+ has not made a decision to increase production levels compared to the plan. These factors have led to a sharp increase in oil prices on the world market in recent times.\n\nThe price of crude oil on October 10, 2021, reached $80.11 per barrel for WTI and $82.58 per barrel for Brent, the highest in the past seven years. The average prices of various oil products in the world in the 15 days before the oil price management period on October 11, 2021, increased significantly 

In [16]:
for value in values:
    print(token_count(value[0]), "tokens")
    print(value[0])
    print(token_count(value[1]), "tokens")
    print(value[1])

448 tokens
The current situation of the oil market is facing significant challenges due to the strong increase in global oil prices. The world oil market has been influenced by the rising demand as economies around the world are gradually recovering. Production and business activities are resuming as many countries are adopting living together, adapting safely to the Covid-19 pandemic. Energy demand, including oil, is also increasing to prepare for the winter season as gas prices surge. Additionally, OPEC+ has not made a decision to increase production levels compared to the plan. These factors have led to a sharp increase in oil prices on the world market in recent times.

The price of crude oil on October 10, 2021, reached $80.11 per barrel for WTI and $82.58 per barrel for Brent, the highest in the past seven years. The average prices of various oil products in the world in the 15 days before the oil price management period on October 11, 2021, increased significantly compared to th

In [5]:
conclusion_function = plugin["Conclusion"]
summContent = " ".join([value[1] for value in values])
conclusion = await kernel.invoke(conclusion_function, sk.KernelArguments(conclusion = outline_dict["conclusion"], summContent = summContent))
print(str(conclusion))


Trong bối cảnh giá dầu tăng mạnh trên thị trường thế giới và ảnh hưởng của đại dịch Covid-19, việc điều chỉnh giá xăng dầu trên thị trường nội địa là cực kỳ quan trọng để duy trì ổn định thị trường và hỗ trợ người tiêu dùng cũng như doanh nghiệp. Quyết định của Chính phủ về việc tăng cường quản lý quỹ ổn định giá nhiên liệu, điều chỉnh giá xăng E5RON92 và RON95 để thúc đẩy bảo vệ môi trường, cũng như đảm bảo sự cân đối giữa các bên tham gia thị trường, là bước đi chiến lược để ứng phó với biến động giá dầu toàn cầu và hỗ trợ phát triển bền vững của nền kinh tế. Dù có những thách thức, nhưng việc đánh giá chính xác và áp dụng biện pháp hiệu quả sẽ giúp thị trường xăng dầu ổn định và phát triển trong tương lai.


In [6]:
print(token_count(str(conclusion)))

354


In [7]:
recommendation_function = plugin["Recommendation"]
summContent = " ".join([value[1] for value in values])
recommendation = await kernel.invoke(recommendation_function, sk.KernelArguments(recommendation = outline_dict["recommendations"], summContent = summContent))
print(str(recommendation))
print(token_count(str(recommendation)))

Based on the analysis presented in the report, we recommend that businesses operating in the oil market sector closely monitor global oil price trends and market fluctuations. It is crucial for companies to adapt their pricing strategies and supply chain management to mitigate the impact of rising oil prices on their operations. Additionally, businesses should consider diversifying their energy sources and exploring alternative fuels to reduce dependency on traditional oil products.

Furthermore, we suggest that companies in the oil market sector collaborate with government agencies and industry associations to advocate for policies that support market stability, consumer protection, and environmental sustainability. By actively engaging in discussions and initiatives aimed at addressing the challenges posed by fluctuating oil prices, businesses can contribute to a more resilient and sustainable energy market.

Moreover, we recommend that businesses in the oil market sector prioritize 